In [1]:
import pandas as pd
import numpy as np

In [2]:
import sys
sys.path.append("../")
from schemas.output_parser import OutputStructure
from langchain_openai import ChatOpenAI
from prompts.PromptMaker import PromptBuilder

from langchain_core.messages import SystemMessage, HumanMessage

from prompts.agent_prompt import AGENT_SYS_PROMPT, HUMAN_PROMPT

In [3]:
llm= ChatOpenAI(model="gpt-4.1", temperature=0.0)
agent = llm.with_structured_output(OutputStructure)


In [4]:
df = pd.read_csv("dataset_C3_clean.csv")
df.head(2)

,fixed,buggy,baseline_ast_gumtree,line_id_tp_dataset
0,"private static void METHOD_1 ( long VAR_1 , lo...",private static native void METHOD_1 ( long VAR...,['Insert Wra'],1
1,"public void METHOD_1 ( TYPE_1 VAR_1 , TYPE_2 V...","public void METHOD_1 ( TYPE_1 VAR_1 , TYPE_2 V...","['Insert Invocation', 'Insert TypeAccess', 'In...",2


In [35]:
limit = 601

fixed = df["fixed"].iloc[limit]
buggy = df["buggy"].iloc[limit]
baseline = df["baseline_ast_gumtree"].iloc[limit]
print(df["fixed"].iloc[limit])
print(df["buggy"].iloc[limit])
print(df["baseline_ast_gumtree"].iloc[limit])
print(df["line_id_tp_dataset"].iloc[limit])

public java.util.Set < TYPE_1 > METHOD_1 ( java.lang.String VAR_1 ) { java.util.Set < TYPE_1 > set = this . VAR_2 . get ( VAR_1 ) ; return set == null ? new java.util.HashSet < TYPE_1 > ( ) : new java.util.HashSet < TYPE_1 > ( set ) ; }
public java.util.Set < TYPE_1 > METHOD_1 ( java.lang.String VAR_1 ) { java.util.Set < TYPE_1 > set = this . VAR_2 . get ( VAR_1 ) ; return set == null ? new java.util.HashSet < TYPE_1 > ( ) : new java.util.HashSet ( set ) ; }
[]
709


In [26]:
all_ops = "['Assignment', 'BinaryOperator', 'Block', 'ConstructorCall', 'FieldRead', 'FieldWrite', 'If', 'Invocation', 'Literal', 'LocalVariable', 'Parameter', 'Return', 'ThisAccess', 'TypeAccess', 'TypeReference', 'UnaryOperator', 'VariableRead', 'Wra']"


human_prompt = HUMAN_PROMPT.format(
    CodeFix=fixed,
    BuggyCode=buggy,
    final_length_ops=18,
    ops_for_gpt = all_ops
)




In [9]:
print(AGENT_SYS_PROMPT)


You are an expert in Abstract Syntax Tree (AST) analysis for the Java language. 
You will be provided with two artifacts: (1) a pair of code snippets (`snippet one`, the fixed version, and `snippet two`, the buggy version), and (2) a set of Java node types titled "ALLOWED_NODE_SET". 
These snippets are a processed representation of Java code where identifiers have been replaced with placeholders.

<YOUR_TASK>
Your task is to identify the precise AST modifications required to transform `snippet one` into `snippet two`. 
The modifications fall into four categories: `Insert`, `Delete`, `Update`, or `Move`.
</YOUR_TASK>

<IMPORTANT_RULES>
- You MUST associate each modification ONLY with a component type from the provided "ALLOWED_NODE_SET".
- A component type from the set can be used more than once if multiple modifications of the same type occurred. 
- The "ALLOWED_NODE_SET" is a list of valid options; you are not required to use every component from this list.
- Your analysis must be ba

In [27]:
final_prompt = [SystemMessage(AGENT_SYS_PROMPT), HumanMessage(human_prompt)]
results = agent.invoke(final_prompt)

In [28]:
results.mut_op_list

['Delete TypeReference']